In [411]:
import pandas as pd
from geopy.geocoders import Nominatim
import time
import numpy as np
from fuzzywuzzy import fuzz
import time
from tqdm import tqdm
tqdm.pandas()

In [412]:
OPT_path=r"C:\Users\khat\OneDrive - PENNSYLVANIA COMPENSATION RATING BUREAU\Desktop\Opti_samp_1000.csv"
DF_pc=pd.read_csv(OPT_path)
DF = DF_pc.dropna(subset=['StreetAddress_Rev', 'City', 'State', 'Zipcode'])
DF.reset_index()

,index,PrimaryInsuredName,StreetAddress,StreetAddress_Rev,City,State,Zipcode,NumberOfEmployees,Exposure
0,0,NATURAL DAIRY PRODUCTS CORPORATION DBA NATURAL...,110 CHRISTINA MEDICAL CENTER,110 CHRISTINA MEDICAL Ctr,NEWARK,DE,19702.0,0.0,1449000.0
1,1,King Kargo Carriers LLC,421 Pierce Run,421 Pierce Run,Newark,DE,19702.0,1.0,40000.0
2,2,MCCUNE FOUNDATION,1195 RIVER RDPO BOX 302,1195 RIVER RDPO BOX 302,MARIETTA,PA,17547.0,0.0,648526.0
3,3,Benner Deer Fence LLC,74 Commerce Drive,74 Commerce Dr,Wyomissing,PA,19610.0,0.0,29506.0
4,4,RICHARD SERVICES INC,PO BOX 1225,PO BOX 1225,HOCKESSIN,DE,19707.0,0.0,39650.0
...,...,...,...,...,...,...,...,...,...
939,995,DEVOTED TO YOU MELISSA CRAWFORD...,232 N BALPH AVE,232 N BALPH AVE,PITTSBURGH,PA,15202.0,0.0,80000.0
940,996,Terrence L Allshouse Jr DBA Allshouse Energy,8720 Stony Point Parkway,8720 Stony Point Pkwy,Rockwood,PA,15557.0,0.0,238362.0
941,997,MATCHMAKERS INC,18 Richboro Rd,18 Richboro Rd,Newtown,PA,18940.0,8.0,145212.0
942,998,OXFORD PRODUCE LLC,5711 OXFORD AVE,5711 OXFORD AVE,PHILADELPHIA,PA,19149.0,0.0,96126.0


In [413]:
print(DF.shape)
DF.head()

(944, 8)


,PrimaryInsuredName,StreetAddress,StreetAddress_Rev,City,State,Zipcode,NumberOfEmployees,Exposure
0,NATURAL DAIRY PRODUCTS CORPORATION DBA NATURAL...,110 CHRISTINA MEDICAL CENTER,110 CHRISTINA MEDICAL Ctr,NEWARK,DE,19702.0,0.0,1449000.0
1,King Kargo Carriers LLC,421 Pierce Run,421 Pierce Run,Newark,DE,19702.0,1.0,40000.0
2,MCCUNE FOUNDATION,1195 RIVER RDPO BOX 302,1195 RIVER RDPO BOX 302,MARIETTA,PA,17547.0,0.0,648526.0
3,Benner Deer Fence LLC,74 Commerce Drive,74 Commerce Dr,Wyomissing,PA,19610.0,0.0,29506.0
4,RICHARD SERVICES INC,PO BOX 1225,PO BOX 1225,HOCKESSIN,DE,19707.0,0.0,39650.0


In [414]:
# List of stop phrases (you can add more as needed)
Astop_phrases = ['#', 'po box', 'hwy', 'unit', 'ste', 'bldg', 'station', 'apt']
Bstop_phrases=['&']
stop_phrases=Astop_phrases
#+Bstop_phrases
# Function to remove everything after stop phrases
def clean_address(address):
    # Convert to lowercase to handle case-insensitive matching
    address_lower = address.lower()
    
    # Check each stop phrase in the list
    for phrase in stop_phrases:
        po_pos = address_lower.find(phrase)  # Find position of the stop phrase
        if po_pos != -1 and phrase in Astop_phrases:
            return address[:po_pos].strip()
    return address  # If no stop phrase is found, return the original address

# Apply the function to the address column
DF['StreetAddress_Rev'] = DF['StreetAddress_Rev'].apply(clean_address)

C:\Users\khat\AppData\Local\Temp\ipykernel_41024\2111623165.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['StreetAddress_Rev'] = DF['StreetAddress_Rev'].apply(clean_address)


In [415]:
#Combine columns
DF['Zipcode'] = DF['Zipcode'].apply(lambda x: str(int(x)) if pd.notnull(x) else x)
DF['Full']= DF['StreetAddress_Rev'].str.strip() + ', ' + DF['City'].str.strip() + ', ' + DF['State'].str.strip() + ' ' + DF['Zipcode'].str.strip()

C:\Users\khat\AppData\Local\Temp\ipykernel_41024\3745672049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Zipcode'] = DF['Zipcode'].apply(lambda x: str(int(x)) if pd.notnull(x) else x)
C:\Users\khat\AppData\Local\Temp\ipykernel_41024\3745672049.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Full']= DF['StreetAddress_Rev'].str.strip() + ', ' + DF['City'].str.strip() + ', ' + DF['State'].str.strip() + ' ' + DF['Zipcode'].str.strip()


In [416]:
DF.head()

,PrimaryInsuredName,StreetAddress,StreetAddress_Rev,City,State,Zipcode,NumberOfEmployees,Exposure,Full
0,NATURAL DAIRY PRODUCTS CORPORATION DBA NATURAL...,110 CHRISTINA MEDICAL CENTER,110 CHRISTINA MEDICAL Ctr,NEWARK,DE,19702,0.0,1449000.0,"110 CHRISTINA MEDICAL Ctr, NEWARK, DE 19702"
1,King Kargo Carriers LLC,421 Pierce Run,421 Pierce Run,Newark,DE,19702,1.0,40000.0,"421 Pierce Run, Newark, DE 19702"
2,MCCUNE FOUNDATION,1195 RIVER RDPO BOX 302,1195 RIVER RD,MARIETTA,PA,17547,0.0,648526.0,"1195 RIVER RD, MARIETTA, PA 17547"
3,Benner Deer Fence LLC,74 Commerce Drive,74 Commerce Dr,Wyomissing,PA,19610,0.0,29506.0,"74 Commerce Dr, Wyomissing, PA 19610"
4,RICHARD SERVICES INC,PO BOX 1225,,HOCKESSIN,DE,19707,0.0,39650.0,", HOCKESSIN, DE 19707"


In [417]:

# # Initialize the geocoder and give it a check
# geolocator = Nominatim(user_agent="Opti_Geo", timeout=12)
# location = geolocator.geocode("3927 NEWBURG RD, EASTON, PA 18045")
# if location != None:
#     print(location.address, location.latitude, location.longitude)
# def geocode_address(address, retries=3, delay=3):
#     for _ in range(retries):
#         try:
#             # Geocode and return a list of results
#             locations = geolocator.geocode(address, exactly_one=False)
            
#             if locations:
#                 # Sort locations based on the 'address' field or any other criteria
#                 best_location = max(locations, key=lambda loc: loc.address.lower() == address.lower())
#                 return best_location
            
#             return None  # If no location matches
            
#         except GeocoderTimedOut:
#             time.sleep(delay)  # Wait before retrying
#     return None  # If geocoding fails after retries 

In [418]:


# Define the geocoder and related functions
geolocator = Nominatim(user_agent="Opti_Geo", timeout=12)

def geocode_address(address, retries=5, delay=5):
    for _ in range(retries):
        try:
            locations = geolocator.geocode(address, exactly_one=False)
            if locations:
                best_location = max(locations, key=lambda loc: fuzz.ratio(address.lower(), loc.address.lower()))
                return best_location
            return None
        except GeocoderTimedOut:
            time.sleep(delay)  
    return None

# Example of geocoding a batch of addresses
addresses = ["1600 PENNSYLVANIA AVE NW, WASHINGTON, DC 20500"]
results = [geocode_address(address) for address in addresses]
for result in results:
    if result:
        print(f"Address: {result.address}, Latitude: {result.latitude}, Longitude: {result.longitude}")
    else:
        print("Geocoding failed")

Address: White House, 1600, Pennsylvania Avenue Northwest, Ward 2, Washington, District of Columbia, 20500, United States, Latitude: 38.897699700000004, Longitude: -77.03655315


In [419]:
sample_test=DF.sample(10)
#random_state=4
sample_test.head()

,PrimaryInsuredName,StreetAddress,StreetAddress_Rev,City,State,Zipcode,NumberOfEmployees,Exposure,Full
907,THE COR PROJECT,209 ROSEMAN CT,209 ROSEMAN CT,NEWARK,DE,19711,1.0,685877.0,"209 ROSEMAN CT, NEWARK, DE 19711"
997,MATCHMAKERS INC,18 Richboro Rd,18 Richboro Rd,Newtown,PA,18940,8.0,145212.0,"18 Richboro Rd, Newtown, PA 18940"
142,LaKirk LLC,452 East St,452 E St,Bloomsburg,PA,17815,0.0,180000.0,"452 E St, Bloomsburg, PA 17815"
203,ELDER PAWS PET SITTING LLC,120 MILBURY RD,120 MILBURY RD,COATESVILLE,PA,19320,0.0,62660.0,"120 MILBURY RD, COATESVILLE, PA 19320"
843,CBRE GROUP INC,100 S INDEPENDENCE MALL W,100 S INDEPENDENCE MALL W,PHILADELPHIA,PA,19106,0.0,285450742.0,"100 S INDEPENDENCE MALL W, PHILADELPHIA, PA 19106"


In [420]:

# # DF[['latitude', 'longitude']] = DF['address'].progress_apply(lambda x: pd.Series(geocode_address(x)))
sample_test[['lat', 'long'] ]= sample_test['Full'].progress_apply(lambda x: pd.Series(geocode_address(x)))


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:11<00:00,  1.14s/it]


In [421]:
sample_test.drop('lat', axis=1, inplace=True)
sample_test.rename(columns={'long':'GeoCode'}, inplace=True)
sample_test.head(10)

,PrimaryInsuredName,StreetAddress,StreetAddress_Rev,City,State,Zipcode,NumberOfEmployees,Exposure,Full,GeoCode
907,THE COR PROJECT,209 ROSEMAN CT,209 ROSEMAN CT,NEWARK,DE,19711,1.0,685877.0,"209 ROSEMAN CT, NEWARK, DE 19711",NaN
997,MATCHMAKERS INC,18 Richboro Rd,18 Richboro Rd,Newtown,PA,18940,8.0,145212.0,"18 Richboro Rd, Newtown, PA 18940","(40.22749502443343, -74.94158686425871)"
142,LaKirk LLC,452 East St,452 E St,Bloomsburg,PA,17815,0.0,180000.0,"452 E St, Bloomsburg, PA 17815","(41.001979, -76.44966765924704)"
203,ELDER PAWS PET SITTING LLC,120 MILBURY RD,120 MILBURY RD,COATESVILLE,PA,19320,0.0,62660.0,"120 MILBURY RD, COATESVILLE, PA 19320",NaN
843,CBRE GROUP INC,100 S INDEPENDENCE MALL W,100 S INDEPENDENCE MALL W,PHILADELPHIA,PA,19106,0.0,285450742.0,"100 S INDEPENDENCE MALL W, PHILADELPHIA, PA 19106","(39.949115, -75.1509)"
744,Veterinarian Partners of Pennsylvania PC,1658 PA # 228,1658 PA 228,CRANBERRY TWP,PA,16066,0.0,962000.0,"1658 PA 228, CRANBERRY TWP, PA 16066","(40.685328461770645, -80.07248121823102)"
216,Univest Financial Corporation,1 Fitzwatertown Rd,1 Fitzwatertown Rd,Willow Grove,PA,19090,4.0,98840052.0,"1 Fitzwatertown Rd, Willow Grove, PA 19090","(40.148851, -75.129358)"
896,HERRING SYSTEMS LTD,41 LONGVIEW DR,41 LONGVIEW DR,NAZARETH,PA,18064,0.0,335960.0,"41 LONGVIEW DR, NAZARETH, PA 18064","(40.75228209167771, -75.29069145311671)"
279,ARIANE SYSTEMS NORTH AMERICA INC,109 PHEASANT RUN,109 PHEASANT RUN,NEWTOWN,PA,18940,0.0,0.0,"109 PHEASANT RUN, NEWTOWN, PA 18940","(40.225534907912504, -74.91634579809165)"
444,Hope Batchelor DVM PC,511 WORTHINGTON RD,511 WORTHINGTON RD,CHESTER SPRINGS,PA,19425,0.0,326165.0,"511 WORTHINGTON RD, CHESTER SPRINGS, PA 19425",NaN


In [422]:
#see if we can automate 1000 runs of 10 samples and get an accuract rate
def API_exp(trials, df):
    Hit_rate=0
    offset= np.random.randint(1, 1000)
    for i in tqdm(range(0, trials), desc='Running Trials'):
        samp=df.sample(10, random_state=offset+i)
        samp[['lat', 'long'] ]= samp['Full'].progress_apply(lambda x: pd.Series(geocode_address(x)))
        Hit_rate += samp['long'].isna().sum() / len(samp) * 100
    Hit_rate= Hit_rate/trials
    print(f"The Average found addresses after {trials} trials is: {100-Hit_rate}")

# API_exp(100, DF)